## Ayudantía 2
#### Regresión Lineal

In [183]:
import pandas as pd
from sklearn.model_selection import train_test_split    
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score # MSE y R2
from sklearn.preprocessing import StandardScaler, LabelEncoder
from prettytable import PrettyTable

In [184]:
df = pd.read_csv('housing_Tarea2.csv')
df

,house_type,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,verificationDate,description,SecurityDeposit,Status
0,1 RK Studio Apartment,400 sq ft,Kalkaji,Delhi,28.545561,77.254349,22000,INR,1.0,NaN,NaN,NaN,Posted a day ago,"Fully furnished, loaded with amenities & gadge...",No Deposit,Furnished
1,1 RK Studio Apartment,400 sq ft,Mansarover Garden,Delhi,28.643259,77.132828,20000,INR,1.0,NaN,NaN,NaN,Posted 9 days ago,Here is an excellent 1 BHK Independent Floor a...,No Deposit,Furnished
2,2 BHK Independent Floor,500 sq ft,Uttam Nagar,Delhi,28.618677,77.053352,8500,INR,1.0,NaN,NaN,NaN,Posted 12 days ago,"Zero Brokerage.\r\n\r\n2 Room set, Govt bijali...",No Deposit,Semi-Furnished
3,3 BHK Independent House,"1,020 sq ft",Model Town,Delhi,28.712898,77.180000,48000,INR,3.0,NaN,NaN,NaN,Posted a year ago,Itâs a 3 bhk independent house situated in M...,No Deposit,Furnished
4,2 BHK Apartment,810 sq ft,Sector 13 Rohini,Delhi,28.723539,77.131424,20000,INR,2.0,NaN,NaN,NaN,Posted a year ago,Well designed 2 bhk multistorey apartment is a...,No Deposit,Unfurnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,4 BHK Villa,"5,896 sq ft",Sunder Nagar,Delhi,28.618437,76.961784,1022001,INR,4.0,2.0,NaN,NaN,Posted 2 months ago,Its four bhk villa in the super location of De...,"40,10,102",Unfurnished
4996,5 BHK Independent House,"6,521 sq ft",Sunder Nagar,Delhi,28.618437,76.961784,1549181,INR,4.0,2.0,NaN,NaN,Posted 2 months ago,A 5 bhk property is available for rent in Sund...,"54,01,015",Unfurnished
4997,3 BHK Independent Floor,"1,855 sq ft",New Friends Colony,Delhi,28.567051,77.273560,301012,INR,3.0,2.0,NaN,NaN,Posted 2 months ago,Its three bhk builder floor in the super locat...,"18,18,181",Unfurnished
4998,3 BHK Independent Floor,"2,856 sq ft",New Friends Colony,Delhi,28.567051,77.273560,301011,INR,3.0,2.0,NaN,NaN,Posted 2 months ago,Its three bhk builder floor in the super locat...,"10,10,110",Unfurnished


# Limpieza de Datos

In [185]:
# Limpieza de datos para la columna house_type
# "mujer" "hombre" <- 0 1 <- encoding, con el tipo de Label
df["numRooms"] = df["house_type"].str[0].astype(int)
# 3 BHK <- 3 Bedroom, 1 Hall, 1 Kitchen
df["house_type"].value_counts()
df["rooms"] = df["house_type"].str.split(" ").str[1]
df["housing_type"] = df["house_type"].str.split(" ").str[2:].str.join(" ")

# Encoding características categóricas
encoder = LabelEncoder()
df["rooms"] = encoder.fit_transform(df["rooms"]).astype(int)
df["housing_type"] = encoder.fit_transform(df["housing_type"]).astype(int)
df.drop(columns=["house_type"], inplace=True)

In [186]:
# Limpieza columna house_size
df["house_size"] = df["house_size"].str.replace(",","") 
df["house_size"] = df["house_size"].str.split(" ").str[0].astype(int)

In [187]:
df["location"] = encoder.fit_transform(df["location"]).astype(int)
df["city"] = encoder.fit_transform(df["city"]).astype(int)
df["currency"] = encoder.fit_transform(df["currency"]).astype(int)
df = df.fillna(0) # Rellenar valores nulos con 0


In [188]:
# Codificar la columna isNegotiable: 'Negotiable' -> 1, 0 -> 0
if 'isNegotiable' in df.columns:
    df['isNegotiable'] = df['isNegotiable'].apply(lambda x: 1 if str(x).strip().lower() == 'negotiable' else 0).astype(int)

#


In [189]:
df["currency"].value_counts()
#Columnas a eliminar city, currency

currency
0    5000
Name: count, dtype: int64

In [190]:
verif_map = {
    'day' : 1, 'days' : 1,
    "week" : 7, "weeks" : 7, 
    "month" : 30, "months" : 30,
    "year" : 365, "years" : 365
}
df[["cant_veces_str", "mult"]] = df["verificationDate"].str.extract(r'Posted\s+(\d+|a|an)\s+(\w+)')
df['cant_veces'] = df['cant_veces_str'].replace({'a': 1, 'an': 1}).astype(int)
df["days_since_verif"] = df["cant_veces"] * df["mult"].map(verif_map) # .apply()
df.drop(columns=["verificationDate", "cant_veces_str", "mult", "cant_veces", "description"], inplace=True)
# Rellenar los NaN de la columna 'days_since_verif' con 0
df['days_since_verif'] = df['days_since_verif'].fillna(0)

df

,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
0,400,88,0,28.545561,77.254349,22000,0,1.0,0.0,0,0.0,No Deposit,Furnished,1,1,3,1.0
1,400,124,0,28.643259,77.132828,20000,0,1.0,0.0,0,0.0,No Deposit,Furnished,1,1,3,9.0
2,500,259,0,28.618677,77.053352,8500,0,1.0,0.0,0,0.0,No Deposit,Semi-Furnished,2,0,1,12.0
3,1020,133,0,28.712898,77.180000,48000,0,3.0,0.0,0,0.0,No Deposit,Furnished,3,0,2,365.0
4,810,201,0,28.723539,77.131424,20000,0,2.0,0.0,0,0.0,No Deposit,Unfurnished,2,0,0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5896,249,0,28.618437,76.961784,1022001,0,4.0,2.0,0,0.0,"40,10,102",Unfurnished,4,0,4,60.0
4996,6521,249,0,28.618437,76.961784,1549181,0,4.0,2.0,0,0.0,"54,01,015",Unfurnished,5,0,2,60.0
4997,1855,146,0,28.567051,77.273560,301012,0,3.0,2.0,0,0.0,"18,18,181",Unfurnished,3,0,1,60.0
4998,2856,146,0,28.567051,77.273560,301011,0,3.0,2.0,0,0.0,"10,10,110",Unfurnished,3,0,1,60.0


In [191]:
#convertir a int la columna status
df["Status"] = encoder.fit_transform(df["Status"]).astype(int)
df

,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
0,400,88,0,28.545561,77.254349,22000,0,1.0,0.0,0,0.0,No Deposit,0,1,1,3,1.0
1,400,124,0,28.643259,77.132828,20000,0,1.0,0.0,0,0.0,No Deposit,0,1,1,3,9.0
2,500,259,0,28.618677,77.053352,8500,0,1.0,0.0,0,0.0,No Deposit,1,2,0,1,12.0
3,1020,133,0,28.712898,77.180000,48000,0,3.0,0.0,0,0.0,No Deposit,0,3,0,2,365.0
4,810,201,0,28.723539,77.131424,20000,0,2.0,0.0,0,0.0,No Deposit,2,2,0,0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5896,249,0,28.618437,76.961784,1022001,0,4.0,2.0,0,0.0,"40,10,102",2,4,0,4,60.0
4996,6521,249,0,28.618437,76.961784,1549181,0,4.0,2.0,0,0.0,"54,01,015",2,5,0,2,60.0
4997,1855,146,0,28.567051,77.273560,301012,0,3.0,2.0,0,0.0,"18,18,181",2,3,0,1,60.0
4998,2856,146,0,28.567051,77.273560,301011,0,3.0,2.0,0,0.0,"10,10,110",2,3,0,1,60.0


In [192]:
# Ordenar y limpiar la columna security_deposit
#PROMPT UTILIZADO: Limpia y ordena la columna "SecurityDeposit" del DataFrame. Convierte los valores "No Deposit" a 0 y, para los valores numéricos separados por comas (por ejemplo, "40,10,102"), suma los números. Asegúrate de que la columna quede como tipo numérico (float).
# 1. Reemplazar 'No Deposit' por 0
# 2. Sumar los valores numéricos separados por comas

def clean_security_deposit(val):
    if isinstance(val, str):
        if val.strip().lower() == 'no deposit':
            return 0
        else:
            # Sumar los valores separados por coma
            return sum([float(x) for x in val.split(',') if x.strip() != ''])
    elif pd.isnull(val):
        return 0
    else:
        return val

# Aplicar la función de limpieza
df['SecurityDeposit'] = df['SecurityDeposit'].apply(clean_security_deposit)
df['SecurityDeposit'] = df['SecurityDeposit'].astype(float)
df

,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
0,400,88,0,28.545561,77.254349,22000,0,1.0,0.0,0,0.0,0.0,0,1,1,3,1.0
1,400,124,0,28.643259,77.132828,20000,0,1.0,0.0,0,0.0,0.0,0,1,1,3,9.0
2,500,259,0,28.618677,77.053352,8500,0,1.0,0.0,0,0.0,0.0,1,2,0,1,12.0
3,1020,133,0,28.712898,77.180000,48000,0,3.0,0.0,0,0.0,0.0,0,3,0,2,365.0
4,810,201,0,28.723539,77.131424,20000,0,2.0,0.0,0,0.0,0.0,2,2,0,0,365.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,5896,249,0,28.618437,76.961784,1022001,0,4.0,2.0,0,0.0,152.0,2,4,0,4,60.0
4996,6521,249,0,28.618437,76.961784,1549181,0,4.0,2.0,0,0.0,70.0,2,5,0,2,60.0
4997,1855,146,0,28.567051,77.273560,301012,0,3.0,2.0,0,0.0,217.0,2,3,0,1,60.0
4998,2856,146,0,28.567051,77.273560,301011,0,3.0,2.0,0,0.0,130.0,2,3,0,1,60.0


In [193]:
df.describe()

,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
count,5000.000000,5000.000000,5000.0,5000.000000,5000.000000,5.000000e+03,5000.0,5000.000000,5000.000000,5000.00000,5000.0,5000.000000,5000.000000,5000.00000,5000.00000,5000.000000,5000.000000
mean,2982.885400,136.314400,0.0,28.578012,77.174499,2.221738e+05,0.0,2.904000,1.069800,0.03580,0.0,107.155200,1.321200,3.09780,0.02760,1.210400,266.971400
std,2168.663368,79.323699,0.0,0.190186,0.115636,2.739843e+05,0.0,1.104458,1.053731,0.18581,0.0,174.287304,0.715914,1.14162,0.16384,1.045347,339.308304
min,150.000000,0.000000,0.0,20.011379,72.771332,3.000000e+03,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,1.00000,0.00000,0.000000,0.000000
25%,1100.000000,65.000000,0.0,28.544489,77.138248,2.950000e+04,0.0,2.000000,0.000000,0.00000,0.0,0.000000,1.000000,2.00000,0.00000,1.000000,30.000000
50%,2500.000000,141.000000,0.0,28.569295,77.196472,1.250000e+05,0.0,3.000000,1.000000,0.00000,0.0,22.000000,1.000000,3.00000,0.00000,1.000000,150.000000
75%,5896.000000,195.000000,0.0,28.618687,77.228950,3.011020e+05,0.0,4.000000,2.000000,0.00000,0.0,218.000000,2.000000,4.00000,0.00000,1.000000,365.000000
max,14521.000000,287.000000,0.0,28.805466,80.361313,3.010101e+06,0.0,10.000000,8.000000,1.00000,0.0,1108.000000,2.000000,9.00000,1.00000,5.000000,1825.000000


In [194]:
scaler = StandardScaler()
df["house_size"] = scaler.fit_transform(df[["house_size"]])


In [195]:
df_baseline = df.copy()
df_baseline.describe()

,house_size,location,city,latitude,longitude,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif
count,5.000000e+03,5000.000000,5000.0,5000.000000,5000.000000,5.000000e+03,5000.0,5000.000000,5000.000000,5000.00000,5000.0,5000.000000,5000.000000,5000.00000,5000.00000,5000.000000,5000.000000
mean,-9.094947e-17,136.314400,0.0,28.578012,77.174499,2.221738e+05,0.0,2.904000,1.069800,0.03580,0.0,107.155200,1.321200,3.09780,0.02760,1.210400,266.971400
std,1.000100e+00,79.323699,0.0,0.190186,0.115636,2.739843e+05,0.0,1.104458,1.053731,0.18581,0.0,174.287304,0.715914,1.14162,0.16384,1.045347,339.308304
min,-1.306412e+00,0.000000,0.0,20.011379,72.771332,3.000000e+03,0.0,0.000000,0.000000,0.00000,0.0,0.000000,0.000000,1.00000,0.00000,0.000000,0.000000
25%,-8.683108e-01,65.000000,0.0,28.544489,77.138248,2.950000e+04,0.0,2.000000,0.000000,0.00000,0.0,0.000000,1.000000,2.00000,0.00000,1.000000,30.000000
50%,-2.226873e-01,141.000000,0.0,28.569295,77.196472,1.250000e+05,0.0,3.000000,1.000000,0.00000,0.0,22.000000,1.000000,3.00000,0.00000,1.000000,150.000000
75%,1.343411e+00,195.000000,0.0,28.618687,77.228950,3.011020e+05,0.0,4.000000,2.000000,0.00000,0.0,218.000000,2.000000,4.00000,0.00000,1.000000,365.000000
max,5.320913e+00,287.000000,0.0,28.805466,80.361313,3.010101e+06,0.0,10.000000,8.000000,1.00000,0.0,1108.000000,2.000000,9.00000,1.00000,5.000000,1825.000000


### Conjunto Limpio

In [196]:
# Crear conjunto limpio
df_limpio = df_baseline.copy()
# Ejemplo de columnas a eliminar:
# Se eliminan columnas irrelevantes o con alta cantidad de nulos, o que no aportan al modelo
columnas_a_eliminar = ['Status', 'SecurityDeposit', 'city', 'currency']
for col in columnas_a_eliminar:
    if col in df_limpio.columns:
        df_limpio.drop(col, axis=1, inplace=True)
# Estandarizar variables numéricas
num_cols = df_limpio.select_dtypes(include=['float64', 'int64']).columns
scaler = StandardScaler()
df_limpio[num_cols] = scaler.fit_transform(df_limpio[num_cols])
df_limpio

,house_size,location,latitude,longitude,price,numBathrooms,numBalconies,isNegotiable,priceSqFt,numRooms,rooms,housing_type,days_since_verif
0,-1.191122,-0.609140,-0.170645,0.690597,-0.730676,-1.724095,-1.015351,-0.192689,0.0,-1.837748,5.935645,1.712139,-0.783942
1,-1.191122,-0.155258,0.343105,-0.360396,-0.737977,-1.724095,-1.015351,-0.192689,0.0,-1.837748,5.935645,1.712139,-0.760362
2,-1.145007,1.546800,0.213840,-1.047752,-0.779954,-1.724095,-1.015351,-0.192689,0.0,-0.961712,-0.168474,-0.201293,-0.751520
3,-0.905204,-0.041787,0.709306,0.047583,-0.635771,0.086929,-1.015351,-0.192689,0.0,-0.085676,-0.168474,0.755423,0.288936
4,-1.002047,0.815545,0.765263,-0.372537,-0.737977,-0.818583,-1.015351,-0.192689,0.0,-0.961712,-0.168474,-1.158009,0.288936
...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1.343411,1.420721,0.212577,-1.839692,2.919537,0.992441,0.882856,-0.192689,0.0,0.790360,-0.168474,2.668854,-0.610041
4996,1.631636,1.420721,0.212577,-1.839692,4.843854,0.992441,0.882856,-0.192689,0.0,1.666396,-0.168474,0.755423,-0.610041
4997,-0.520135,0.122114,-0.057638,0.856745,0.287776,0.086929,0.882856,-0.192689,0.0,-0.085676,-0.168474,-0.201293,-0.610041
4998,-0.058514,0.122114,-0.057638,0.856745,0.287772,0.086929,0.882856,-0.192689,0.0,-0.085676,-0.168474,-0.201293,-0.610041


### Conjunto con Interacción

In [197]:
# Crear conjunto con interacción
df_interaccion = df_baseline.copy()
df_interaccion['lat_long_interaction'] = df_interaccion['latitude'] * df_interaccion['longitude']
# Por ejemplo: Se elimina 'latitude' y 'longitude' porque su información ya está contenida en la interacción
df_interaccion.drop(['latitude', 'longitude'], axis=1, inplace=True)
df_interaccion

,house_size,location,city,price,currency,numBathrooms,numBalconies,isNegotiable,priceSqFt,SecurityDeposit,Status,numRooms,rooms,housing_type,days_since_verif,lat_long_interaction
0,-1.191122,88,0,22000,0,1.0,0.0,0,0.0,0.0,0,1,1,3,1.0,2205.268712
1,-1.191122,124,0,20000,0,1.0,0.0,0,0.0,0.0,0,1,1,3,9.0,2209.335567
2,-1.145007,259,0,8500,0,1.0,0.0,0,0.0,0.0,1,2,0,1,12.0,2205.165014
3,-0.905204,133,0,48000,0,3.0,0.0,0,0.0,0.0,0,3,0,2,365.0,2216.061496
4,-1.002047,201,0,20000,0,2.0,0.0,0,0.0,0.0,2,2,0,0,365.0,2215.487491
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1.343411,249,0,1022001,0,4.0,2.0,0,0.0,152.0,2,4,0,4,60.0,2202.525962
4996,1.631636,249,0,1549181,0,4.0,2.0,0,0.0,70.0,2,5,0,2,60.0,2202.525962
4997,-0.520135,146,0,301012,0,3.0,2.0,0,0.0,217.0,2,3,0,1,60.0,2207.477712
4998,-0.058514,146,0,301011,0,3.0,2.0,0,0.0,130.0,2,3,0,1,60.0,2207.477712


In [198]:
print(df.isnull().sum())  # Muestra cuántos NaN hay por columna

house_size          0
location            0
city                0
latitude            0
longitude           0
price               0
currency            0
numBathrooms        0
numBalconies        0
isNegotiable        0
priceSqFt           0
SecurityDeposit     0
Status              0
numRooms            0
rooms               0
housing_type        0
days_since_verif    0
dtype: int64


### Modelos de Regresión Lineal
A continuación se ajustan tres modelos de regresión lineal múltiple, uno para cada conjunto: baseline, limpio e interacción. Se comparan sus resultados en una tabla.

In [199]:
# Función para entrenar y evaluar un modelo
def entrenar_evaluar(df, nombre):
    X = df.drop(columns=["price"])
    y = df["price"]
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=254)
    modelo = LinearRegression()
    modelo.fit(X_train, y_train)
    y_pred = modelo.predict(X_test)
    mse = mean_squared_error(y_test, y_pred)
    r2 = r2_score(y_test, y_pred)
    return nombre, mse, r2

# Entrenar y evaluar los tres modelos
resultados = []
resultados.append(entrenar_evaluar(df_baseline, "Baseline"))
resultados.append(entrenar_evaluar(df_limpio, "Limpio"))
resultados.append(entrenar_evaluar(df_interaccion, "Limpio + Interacción"))

# Mostrar resultados en tabla
from prettytable import PrettyTable
table = PrettyTable()
table.field_names = ["Modelo", "MSE", "R2"]
for nombre, mse, r2 in resultados:
    table.add_row([nombre, mse, r2])
print(table)

+----------------------+---------------------+--------------------+
|        Modelo        |         MSE         |         R2         |
+----------------------+---------------------+--------------------+
|       Baseline       |  36080126335.050964 | 0.576838557294774  |
|        Limpio        | 0.49054782171352057 | 0.5681987742871475 |
| Limpio + Interacción |  36158420247.484955 | 0.5759202965150599 |
+----------------------+---------------------+--------------------+


In [200]:
# Variables
X = df_baseline.drop(columns=["price"])  # Variables independientes
y = df_baseline["price"]                 # Variable dependiente

# División del conjunto
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.8, random_state=254)
 

In [201]:
# Ajuste/Entrenamiento del modelo
lr = LinearRegression()
lr.fit(X_train, y_train)
y_prediction = lr.predict(X_test) # y_gorrito

In [202]:
mse = mean_squared_error(y_test, y_prediction)
r2 = r2_score(y_test, y_prediction)

In [203]:
table = PrettyTable()
table.field_names = ["Modelo", "MSE", "R2"]
table.add_row(["Baseline", mse, r2])
table.add_row(["Limpio", 0, 0])
table.add_row(["Limpio + interacción", 0, 0])
table

Modelo,MSE,R2
Baseline,36080126335.050964,0.576838557294774
Limpio,0,0
Limpio + interacción,0,0
